In [1]:
import torch
import gc
torch.cuda.empty_cache()
gc.collect()


0

In [1]:
from setup import setup_src_path
print(setup_src_path())
import data.processed as processed
import config.config as config
import utils.setup as setup
import utils.functions as fn
from importlib import reload



['/home/guest/Desktop/projects/intial-experments/domain_adaptation_project/conference-experment', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages', '/home/guest/Desktop/projects/intial-experments/domain_adaptation_project/modules']


/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
2024-05-01 00:20:01.816973: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-01 00:20:02.023371: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFl

In [2]:
from datasets import load_from_disk

source_data=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/source_data")
source_data_eval=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/source_data_eval")
target_data=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/target_data")
target_data_eval=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/target_data_eval")
test_target_data=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/test_target_data")
unsupervised_target=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/unsupervised_target")


In [3]:
from torch.utils.data import DataLoader
source_data_loader = DataLoader(source_data, batch_size=32, shuffle=True)
target_data_loader = DataLoader(target_data, batch_size=32)
source_data_eval_loader = DataLoader(source_data_eval, batch_size=32, shuffle=True)
target_data_eval_loader = DataLoader(target_data_eval, batch_size=32)

target_test_loader = DataLoader(test_target_data, batch_size=32)

In [4]:
from adapters import AutoAdapterModel,init
from tqdm import tqdm

from transformers import AutoConfig, AutoModelForMaskedLM

mdlcfg = AutoConfig.from_pretrained(
    config.Config.MODEL_NAME,
 
)
model = AutoModelForMaskedLM.from_pretrained(
    config.Config.MODEL_NAME,
)
reload(fn)
fn.print_trainable_parameters(model)

trainable params: 66985530 || all params: 66985530 || trainable%: 100.0


In [5]:
init(model)

In [6]:
from transformers import AdamW
from adapters import AdapterConfig
from adapters.composition import Stack

adapter_name= "unipelt_target_mlm"
adapter_config = AdapterConfig.load("unipelt")
model.add_adapter(adapter_name, config=adapter_config)
#model.add_masked_lm_head(
   # adapter_name,
 # )
model.train_adapter(adapter_name)
model.active_adapters = adapter_name

fn.print_trainable_parameters(model)

trainable params: 6334520 || all params: 72727922 || trainable%: 8.709887242481642


In [7]:
model.active_adapters

Stack[unipelt_target_mlm]

In [8]:
model.adapter_summary(as_dict=True)

[{'name': 'unipelt_target_mlm',
  'architecture': 'union',
  'active': True,
  '#param': 5742392,
  'train': True,
  '%param': 8.653018072753925},
 {'name': 'Full model', '#param': 66362880, '%param': 100.0, 'train': False}]

In [9]:
from transformers import AutoTokenizer

reload(processed)
tokenizer = AutoTokenizer.from_pretrained(config.Config.TOKENIZER_NAME)

tokenized_dataset= processed.tokenize_dataset(unsupervised_target,tokenizer)
tokenized_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'word_ids'],
    num_rows: 52350
})

In [10]:
chunk_size = 128
# Slicing produces a list of lists for each feature
tokenized_samples = tokenized_dataset[444:490]

#for idx, sample in enumerate(tokenized_samples["input_ids"]):
    #print(f"'>>> Review {idx} length: {len(sample)}'")

In [11]:
results = fn.group_texts(tokenized_samples, chunk_size)
for chunk in results["labels"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'


In [12]:
lm_datasets = tokenized_dataset.map(fn.group_texts, batched=True,fn_kwargs={'chunk_size': chunk_size})
lm_datasets

Dataset({
    features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
    num_rows: 18436
})

In [13]:
print(tokenizer.decode(lm_datasets[1]["input_ids"]))
print(tokenizer.decode(lm_datasets[1]['labels']))
print(len(lm_datasets[-1]['labels']))


1874 exhibition of impressionism. [SEP] pissarro never walked on the boulevard des capucines. [SEP] [CLS] a church was once built over this spot, but today the minaret of a small mosque is the best landmark for finding the pool. [SEP] just look for the old church to find the pool. [SEP] [CLS] room 13 contains a number of painted minoan sarcophagi, many decorated with the images of fish or birds. [SEP] the minoans used images of birds and fish to represent how they thought they would be reincarnated. [SEP] [CLS] shinjuku has tokyo's largest discount camera stores, although these rarely
1874 exhibition of impressionism. [SEP] pissarro never walked on the boulevard des capucines. [SEP] [CLS] a church was once built over this spot, but today the minaret of a small mosque is the best landmark for finding the pool. [SEP] just look for the old church to find the pool. [SEP] [CLS] room 13 contains a number of painted minoan sarcophagi, many decorated with the images of fish or birds. [SEP] the

In [14]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
samples = [lm_datasets[i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")
dd=data_collator(samples)
for chunk in dd["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



'>>> [CLS] foreigners from the entire ottoman empire flooded into galata and pera [MASK] attracted by the wealth and [MASK]phistic [MASK] [MASK] the capital. [SEP]ako [MASK] came from other lands to galata. [SEP] [CLS] muslim rule was largely tolerant, and [MASK] peacefully for nearly four [MASK] with a joint christian - muslim protectorate of holy places. [SEP] christians and muslims were fierce [MASK], especially when it [MASK] to holy places. [SEP] [CLS] [MASK] [MASK] des cap [MASK] [MASK], retrace the footsteps of renoir, manet, and piss [MASK]ro as they took their paintings to nadar's house, signposted at number 35, for the historic'

'>>> 1874 exhibition of impression [MASK] [MASK] [SEP] pissarro never walked on the boulevard des capucines. [SEP] [CLS] a church was once built over this spot, [MASK] today the minaret of a small maintaining is the best landmark avoiding finding the pool. [SEP] just look for the old church to [MASK] [MASK] pool. [SEP] [CLS] room 13 contains a numbe

In [15]:
reload(fn)
samples = [lm_datasets[i] for i in range(2)]
batch = fn.whole_word_masking_data_collator(samples,tokenizer)

for chunk in batch["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] foreigners from the entire ottoman empire flooded into galata and [MASK] [MASK], attracted by the wealth [MASK] sophistication of the capital. [SEP] no foreigners [MASK] from other lands to galata. [SEP] [CLS] muslim [MASK] was [MASK] tolerant, and continued peacefully for nearly four [MASK] with a joint christian - muslim protectorate of [MASK] places. [SEP] christians [MASK] muslims [MASK] fierce enemies, especially when it came to holy places. [SEP] [CLS] on boulevard des capucines, retrace the footsteps of renoir, manet, and pissarro as they took [MASK] paintings to [MASK] [MASK]'[MASK] house, signposted at [MASK] 35, for the historic'

'>>> 1874 exhibition of impressionism. [SEP] pissarro never walked on the boulevard des [MASK] [MASK] [MASK]. [SEP] [CLS] a church was once [MASK] over this [MASK], but today the minaret of a small mosque is the best landmark for finding the pool [MASK] [SEP] just look for the old [MASK] to [MASK] the pool. [SEP] [CLS] room 13 contains [

In [16]:

downsampled_dataset = lm_datasets.train_test_split(
    test_size=0.1, seed=42
)
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 16592
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 1844
    })
})

In [17]:

reload(fn)
trainer = fn.train_mlm_model(model,adapter_name,data_collator,tokenizer, downsampled_dataset['train'],downsampled_dataset['test'])

/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [18]:
import math

eval_results = trainer.evaluate()

print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

  0%|          | 0/58 [00:00<?, ?it/s]

>>> Perplexity: 21.81


In [19]:
trainer.train()

  0%|          | 0/5190 [00:00<?, ?it/s]

{'loss': 2.5238, 'learning_rate': 9.961620469083156e-05, 'epoch': 1.0}


  0%|          | 0/58 [00:00<?, ?it/s]

{'eval_loss': 2.13680100440979, 'eval_runtime': 2.2593, 'eval_samples_per_second': 816.178, 'eval_steps_per_second': 25.672, 'epoch': 1.0}
{'loss': 2.2098, 'learning_rate': 8.857142857142857e-05, 'epoch': 2.0}


  0%|          | 0/58 [00:00<?, ?it/s]

...

{'eval_loss': 1.9244098663330078, 'eval_runtime': 2.285, 'eval_samples_per_second': 806.986, 'eval_steps_per_second': 25.382, 'epoch': 9.0}
{'loss': 2.0345, 'learning_rate': 2.5586353944562905e-07, 'epoch': 9.98}


  0%|          | 0/58 [00:00<?, ?it/s]

Overwriting existing adapter 'unipelt_target_mlm'.


{'eval_loss': 1.9278106689453125, 'eval_runtime': 2.277, 'eval_samples_per_second': 809.822, 'eval_steps_per_second': 25.472, 'epoch': 10.0}
{'train_runtime': 436.9699, 'train_samples_per_second': 379.706, 'train_steps_per_second': 11.877, 'train_loss': 2.126393587557107, 'epoch': 10.0}


TrainOutput(global_step=5190, training_loss=2.126393587557107, metrics={'train_runtime': 436.9699, 'train_samples_per_second': 379.706, 'train_steps_per_second': 11.877, 'train_loss': 2.126393587557107, 'epoch': 10.0})

In [20]:
import math

eval_results = trainer.evaluate()

print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

  0%|          | 0/58 [00:00<?, ?it/s]

>>> Perplexity: 6.85


In [21]:
import config.config as config

trainer.model.save_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/{adapter_name}", adapter_name)

In [26]:
import torch
device = torch.device("cpu" if torch.cuda.is_available() else "cpu")
model.to(device)

from transformers import pipeline

mask_filler = pipeline("fill-mask", model=trainer.model, tokenizer=tokenizer)

In [29]:
masked_sentence = "I cancelled my [MASK]."
preds = mask_filler(masked_sentence)

for pred in preds:
    print(f">>> {pred['sequence']}")

>>> i cancelled my plans.
>>> i cancelled my tour.
>>> i cancelled my work.
>>> i cancelled my contract.
>>> i cancelled my job.
